# **Assignment 2 For Clustering Data Analytics**
<hr>

**Submitted by:** Jude M. Ando & Niles Vincent Gabrielle Rondez <br>
**Course & Year:** BSCS - 2 <br>
**Schedule:** M W 10:30 AM - 12:00 PM <br>
<hr>

## Introduction?

## Converted SEE.R Codes into Python Codes

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import gaussian_kde
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from datetime import datetime

def preprocess_data(df, atc_code):
    df = df[df['ATC'] == atc_code].copy()
    df = df.sort_values(by=['pnr', 'eksd'])
    df['prev_eksd'] = df.groupby('pnr')['eksd'].shift(1)
    df = df.dropna()
    df['event_interval'] = (df['eksd'] - df['prev_eksd']).dt.days
    return df

def empirical_cdf(df):
    ecdf = np.sort(df['event_interval'].values)
    y_vals = np.arange(1, len(ecdf) + 1) / len(ecdf)
    return ecdf, y_vals

def plot_ecdf(df):
    ecdf, y_vals = empirical_cdf(df)
    plt.figure(figsize=(10, 5))
    plt.plot(ecdf, y_vals, marker='.', linestyle='none', label='ECDF')
    plt.xlabel('Event Interval (Days)')
    plt.ylabel('ECDF')
    plt.title('Empirical CDF of Event Intervals')
    plt.legend()
    plt.show()

def determine_clusters(df):
    data = np.log(df[['event_interval']].values)
    data = StandardScaler().fit_transform(data)
    inertia = []
    for k in range(1, 10):
        kmeans = KMeans(n_clusters=k, random_state=42)
        kmeans.fit(data)
        inertia.append(kmeans.inertia_)
    
    optimal_k = np.argmax(np.diff(inertia)) + 2  # Heuristic for optimal clusters
    return optimal_k

def kmeans_clustering(df, n_clusters):
    data = np.log(df[['event_interval']].values)
    data = StandardScaler().fit_transform(data)
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    df['cluster'] = kmeans.fit_predict(data)
    return df

def visualize_clusters(df):
    sns.boxplot(x='cluster', y='event_interval', data=df)
    plt.title('Clustering of Event Intervals')
    plt.show()

def sessa_empirical_estimator(df, atc_code):
    df = preprocess_data(df, atc_code)
    plot_ecdf(df)
    n_clusters = determine_clusters(df)
    df = kmeans_clustering(df, n_clusters)
    visualize_clusters(df)
    return df
# Example usage (data needs to be loaded first)
# df = pd.read_csv("your_data.csv")
# df['eksd'] = pd.to_datetime(df['eksd'])
# result = sessa_empirical_estimator(df, "medA")


## Performing Sessa Empircal Estimator on *Data Set Name*

In [2]:
# I am code

### Insights

## Substituting Sessa Empircal Estimator with *Chosen Different Clustering Algorithm* on *Chosen Data Set*

In [ ]:
# I am code

### Insights

## Comparison Between Sessa Empirical Estimator using K-Means, and Sessa Empirical Estimator using *the clustering algorithm of your choice*

text maybe?